In [1]:
import pandas as pd
import numpy as np

In [2]:
OScolnames = {'object_number': 'category',
              'obs_type': 'category',
              'measure_type': 'category',
              '?': 'category',
              'year': np.uint16,
              'month': np.uint8,
              'day': np.float32,
              'date_accuracy': np.float32,
              'RA_HH': np.uint8,
              'RA_mm': np.uint8,
              'RA_ss': np.float32,
              'RA_accuracy': np.float32,
              'RA_RMS': np.float32,
              #'RA_F': 'category',
              'RA_bias': np.float32,
              'RA_delta': np.float32,
              'DEC_HH': np.uint8,
              'DEC_mm': np.uint8,
              'DEC_ss': np.float32,
              'DEC_accuracy': np.float32,
              'DEC_RMS': np.float32,
              #'DEC_F': 'category',
              'DEC_bias': np.float32,
              'DEC_delta': np.float32,
              'MAG': np.float32,
              'MAG_B': 'category',
              'MAG_RMS': np.float32,
              'MAG_resid': np.float32,
              'catalog': 'category',
              'obs_code': 'category',
              'xhi': np.float32,
              'acceptance': 'bool',
              'mag_acceptance': 'bool'}
c=[(0, 11), (11, 13), (13, 15), (15, 17), (17, 21), (22, 25), (25, 40), (40, 50), (50, 53), (53, 56), (56, 64), (64, 77), (77, 83), (87, 94), (96, 103), (103, 107), (107, 110), (110, 117), (117, 130), (128, 136), (141, 148), (149, 156), (156, 161), (161, 164), (164, 170), (170, 178), (178, 180), (180, 186), (189, 194), (194, 196), (196, 197)]

OSdata = pd.read_fwf(r'Data\NEODYS_cleaned.txt', header=None, names=list(OScolnames.keys()), dtype={'obs_code': 'category', 'measure_type': 'category'}, colspecs=c, low_memory=True, index_col=False, usecols=['year', 'obs_code', 'acceptance', 'measure_type', 'RA_delta', 'DEC_delta', 'xhi'], error_bad_lines=False)

---
# Statistics per observation type

In [3]:
types_dict = {'C':'CCD',
              'S':'Space observation',
              'A':'Observations from B1950.0 converted to J2000.',
              'c':'Corrected without republication CCD observation',
              'P': 'Photographic',
              'T':'Meridian or transit circle ',
              'X':'Discovery observation ',
              'x':'Discovery observation ',
              'M':'Micrometer ',
              'H':'Hipparcos geocentric observation ',
              'R':'Radar Observation',
              'E':'Occultations derived observation ',
              'V':'Roving observer observation',
              'n':'Video frames',
              'e':'Encoder ' }

In [4]:
def get_accuracy(typ):
    df = OSdata[(OSdata['acceptance']==True) & (OSdata['measure_type'] == typ) & (OSdata['xhi'] < 3) & (OSdata['RA_delta'] < 100)]
    return np.sqrt(np.power(df.xhi, 2).sum() / len(df))

In [5]:
types = list(OSdata['measure_type'].value_counts().index)
table = pd.DataFrame(index=types, columns=['description', 'count', 'percentage', 'acceptance', 'accuracy', 'timespan'])
for t in types:
    description = types_dict[t]
    count = len(OSdata[(OSdata['measure_type'] == t)].index)
    percentage = count / len(OSdata.index) * 100
    acceptance = (len(OSdata[(OSdata['measure_type'] == t) & (OSdata['acceptance']==True) & (OSdata['xhi'] < 3) & (OSdata['RA_delta'] < 100)]) / count * 100) if count > 0 else 'x'
    accuracy = get_accuracy(t)
    timespan = str(OSdata[OSdata['measure_type'] == t].year.min()) + '-' + str(OSdata[OSdata['measure_type'] == t].year.max())
    table.loc[t] = pd.Series({'description':description, 'count':count, 'percentage':percentage, 'acceptance':acceptance, 'accuracy':accuracy, 'timespan':timespan})

In [6]:
table

,description,count,percentage,acceptance,accuracy,timespan
C,CCD,2444155,97.193,99.2423,0.72903,1990-2020
S,Space observation,44493,1.76929,99.8651,0.849403,1994-2020
A,Observations from B1950.0 converted to J2000.,14125,0.561688,98.7257,0.84234,1893-1998
c,Corrected without republication CCD observation,5308,0.211075,97.2494,1.02603,1991-2013
P,Photographic,3955,0.157273,99.823,0.713469,1898-2017
n,Video frames,1736,0.0690329,99.8848,0.383063,2009-2020
M,Micrometer,642,0.0255294,99.8442,0.850922,1898-1935
V,Roving observer observation,134,0.00532858,86.5672,0.741251,2001-2020
x,Discovery observation,101,0.00401631,100,0.00959579,2010-2010
T,Meridian or transit circle,59,0.00234616,100,0.968745,1988-1995


---
# Statistics per space telescope

In [7]:
def get_space_accuracy(code):
    df = OSdata[(OSdata['acceptance']==True) & (OSdata['measure_type'] == 'S') & (OSdata['obs_code']==code) & (OSdata['xhi'] < 3) & (OSdata['RA_delta'] < 100) ]
    return np.sqrt(np.power(df.xhi, 2).sum() / len(df))

In [8]:
space_telescopes = {'Spitzer Space Telescope':'245', 
                    'Hubble Space Telescope':'250', 
                    'Gaia':'258', 
                    'WISE':'C51', 
                    'NEOSSat':'C53', 
                    'Kepler':'C55', 
                    'TESS':'C57'}
ST_df = pd.DataFrame(index=space_telescopes.keys(), columns=['count', 'acceptance', 'accuracy'])
for name, obs in space_telescopes.items():
    count = len(OSdata[(OSdata['obs_code']==obs) & (OSdata['measure_type']=='S')])
    acceptance = (len(OSdata[(OSdata['obs_code']==obs) & (OSdata['acceptance']==True) & (OSdata['measure_type']=='S')]) / count * 100) if count > 0 else 'x'
    accuracy = get_space_accuracy(obs)
    ST_df.loc[name] = pd.Series({'count':count, 'acceptance':acceptance, 'accuracy':accuracy})
ST_df


,count,acceptance,accuracy
Spitzer Space Telescope,0,x,NaN
Hubble Space Telescope,2,100,2.40075
Gaia,0,x,NaN
WISE,39479,99.9823,0.783923
NEOSSat,3,100,1.56667
Kepler,16,100,0.729576
TESS,4993,99.98,1.25043


In [9]:
#table.to_csv(r'outData\obs_type_stats.csv')
#ST_df.to_csv(r'outData\space_telescope_stats.csv')

In [10]:
table.to_clipboard()

In [11]:
ST_df.to_clipboard()